#### Datasets

In [4]:
import pandas as pd

# Load dataset (update path if needed)
df = pd.read_csv("IMDB_Dataset.csv")

print(df.head())
print(df['sentiment'].value_counts())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


#### Text Preprocessing

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"[^a-zA-Z]", " ", text)  # keep only letters
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

df['cleaned_review'] = df['review'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Train-Test Split & Feature Extraction

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = df['cleaned_review']
y = df['sentiment'].map({'positive':1, 'negative':0})  # convert to 0/1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


#### Train Models

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)

# Naïve Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# SVM
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)


LinearSVC()

#### Model Evaluation

In [8]:
from sklearn.metrics import accuracy_score, f1_score

models = {'Logistic Regression': lr, 'Naive Bayes': nb, 'SVM': svm}

for name, model in models.items():
    y_pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"{name} - Accuracy: {acc:.4f}, F1: {f1:.4f}")


Logistic Regression - Accuracy: 0.8890, F1: 0.8913
Naive Bayes - Accuracy: 0.8546, F1: 0.8566
SVM - Accuracy: 0.8826, F1: 0.8847


#### Test Custom Reviews with a Simple Interface

In [22]:
def predict_sentiment(text, model=svm):
    text = preprocess(text)
    text_vec = vectorizer.transform([text])
    pred = model.predict(text_vec)[0]
    return "Positive 😀" if pred == 1 else "Negative 😡"

# Example test
Review = (input("---(@ Enter your review about the moive:)"))
test = predict_sentiment(Review)
print()
print(f"Review: {Review}:     \n----(@  {test}  @)---")




Review: the movie was good:     
----(@  Positive 😀  @)---
